In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys 
sys.path.append('../maths/') 
from baseball_stats import BasicHitting


In [2]:
k = ['playerID', 'birthYear', 'birthMonth', 'birthDay']
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1', usecols=k)
import_batting = pd.read_csv('../datafiles/Batting.csv', encoding='latin-1')
people = import_people.copy()

#fix fucky dates
people['birthYear'] = people['birthYear'].fillna(1875).astype(int)
people['birthMonth'] = people['birthMonth'].fillna(1).astype(int)
people['birthDay'] = people['birthDay'].fillna(1).astype(int)

#create a birthdate column so we can calulate an age at start of season
people['birthdate'] = people.apply(lambda x: f"""{x['birthYear']}-{x['birthMonth']}-{x['birthDay']}""", axis=1)
people['birthdate'] = pd.to_datetime(people['birthdate'], errors='coerce')

#merge the people and batting dataframes
batting = people.copy().merge(import_batting, on='playerID', how ='inner')

#make sure the raw stats are 0 and not null
batting.loc[:, ['G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP']].fillna(0, inplace=True)

In [ ]:
#TODO: create as function to do career and either 162 game or 600 pa stats
player_batting = batting.copy()
player_batting = player_batting.drop(columns=['stint', 'teamID', 'lgID', 'birthYear', 'birthMonth', 'birthDay', 'G_old', 'G_batting'], axis=1).groupby(['playerID', 'birthdate', 'yearID']).sum().reset_index()
#estimate the start of the season at april 1
player_batting['season_start'] = player_batting.apply(lambda x: f"""{x['yearID']}-04-01""", axis=1)
player_batting['season_start'] = pd.to_datetime(player_batting['season_start'], errors='coerce')

#calculate the age of the player at the start of the season
player_batting['age'] = player_batting.apply(lambda x: relativedelta(x['season_start'], x['birthdate']).years, axis=1)
player_batting.drop(columns=['birthdate', 'season_start'], inplace=True)
player_batting['rowType'] = 'season'
player_batting.rename(columns={'3B':'B3', '2B':'B2', 'GIDP':'GiDP'}, inplace=True)

#adding shit up
player_batting['PA'] = BasicHitting.calc_pa(player_batting)
player_batting['B1'] = BasicHitting.calc_b1(player_batting)
player_batting['TB'] = BasicHitting.calc_tb(player_batting)
player_batting['SBA'] = BasicHitting.calc_sba(player_batting)
player_batting['XBH'] = BasicHitting.calc_xbh(player_batting)
player_batting['TOB'] = BasicHitting.calc_tob(player_batting)
player_batting['RP'] = BasicHitting.calc_rp(player_batting)


,playerID,yearID,G,AB,R,H,B2,B3,HR,RBI,...,GiDP,age,PA,B1,TB,SBA,XBH,TOB,RP,rowType
0,aardsda01,9,331,4,0,0,0,0,0,0.0,...,0.0,27.0,5.0,0,0,0.0,0,0.0,0.0,career
1,aaronha01,23,3298,12364,2174,3771,624,98,755,2297.0,...,328.0,31.0,13940.0,2294,6856,313.0,1477,5205.0,3716.0,career
2,aaronto01,7,437,944,102,216,42,6,13,94.0,...,36.0,27.0,1045.0,155,309,17.0,61,302.0,183.0,career
3,aasedo01,13,448,5,0,0,0,0,0,0.0,...,0.0,29.0,5.0,0,0,0.0,0,0.0,0.0,career
4,abadan01,3,15,21,1,2,0,0,0,0.0,...,1.0,30.0,25.0,2,2,1.0,0,6.0,1.0,career
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20725,zupofr01,3,16,18,3,3,1,0,0,0.0,...,0.0,19.0,20.0,2,4,0.0,1,5.0,3.0,career
20726,zuvelpa01,9,209,491,41,109,17,2,2,20.0,...,8.0,27.0,545.0,88,136,2.0,21,145.0,59.0,career
20727,zuverge01,8,266,142,5,21,2,1,0,7.0,...,3.0,30.0,167.0,18,25,1.0,3,30.0,12.0,career
20728,zwilldu01,4,366,1280,167,364,76,15,30,202.0,...,0.0,25.0,1443.0,243,560,46.0,121,496.0,339.0,career


In [ ]:
#create rows for career
career_df = player_batting.copy()
career_df = career_df.drop(['rowType'], axis=1).groupby(['playerID']).agg({
    'yearID': 'count',
    'G': 'sum',
    'AB': 'sum',
    'R': 'sum',
    'H': 'sum',
    'B2': 'sum',
    'B3': 'sum',
    'HR': 'sum',
    'RBI': 'sum',
    'SB': 'sum',
    'CS': 'sum',
    'BB': 'sum',
    'SO': 'sum',
    'IBB': 'sum',
    'HBP': 'sum',
    'SH': 'sum',
    'SF': 'sum',
    'GiDP': 'sum',
    'age': 'mean',
    'PA': 'sum',
    'B1': 'sum',
    'TB': 'sum',
    'SBA': 'sum',
    'XBH': 'sum',
    'TOB': 'sum',
    'RP': 'sum'
}).reset_index()

career_df['age'] = career_df['age'].round(0).astype(int)
career_df['rowType'] = 'career'

career_df

In [ ]:
#create rows for season average
season_avg_df = player_batting.copy()
season_avg_df = season_avg_df.drop(['rowType'], axis=1).groupby(['playerID']).agg({
    'yearID': 'count',
    'G': 'mean',
    'AB': 'mean',
    'R': 'mean',
    'H': 'mean',
    'B2': 'mean',
    'B3': 'mean',
    'HR': 'mean',
    'RBI': 'mean',
    'SB': 'mean',
    'CS': 'mean',
    'BB': 'mean',
    'SO': 'mean',
    'IBB': 'mean',
    'HBP': 'mean',
    'SH': 'mean',
    'SF': 'mean',
    'GiDP': 'mean',
    'age': 'mean',
    'PA': 'mean',
    'B1': 'mean',
    'TB': 'mean',
    'SBA': 'mean',
    'XBH': 'mean',
    'TOB': 'mean',
    'RP': 'mean'
}).round(2).reset_index()

season_avg_df['age'] = season_avg_df['age'].round(0).astype(int)
season_avg_df['rowType'] = 'seasAvg'

season_avg_df

,playerID,yearID,G,AB,R,H,B2,B3,HR,RBI,...,GiDP,age,PA,B1,TB,SBA,XBH,TOB,RP,rowType
0,aardsda01,9,36.78,0.44,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,27.0,0.56,0.00,0.00,0.00,0.00,0.00,0.00,seasAvg
1,aaronha01,23,143.39,537.57,94.52,163.96,27.13,4.26,32.83,99.87,...,14.26,31.0,606.09,99.74,298.09,13.61,64.22,226.30,161.57,seasAvg
2,aaronto01,7,62.43,134.86,14.57,30.86,6.00,0.86,1.86,13.43,...,5.14,27.0,149.29,22.14,44.14,2.43,8.71,43.14,26.14,seasAvg
3,aasedo01,13,34.46,0.38,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,29.0,0.38,0.00,0.00,0.00,0.00,0.00,0.00,seasAvg
4,abadan01,3,5.00,7.00,0.33,0.67,0.00,0.00,0.00,0.00,...,0.33,30.0,8.33,0.67,0.67,0.33,0.00,2.00,0.33,seasAvg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20725,zupofr01,3,5.33,6.00,1.00,1.00,0.33,0.00,0.00,0.00,...,0.00,19.0,6.67,0.67,1.33,0.00,0.33,1.67,1.00,seasAvg
20726,zuvelpa01,9,23.22,54.56,4.56,12.11,1.89,0.22,0.22,2.22,...,0.89,27.0,60.56,9.78,15.11,0.22,2.33,16.11,6.56,seasAvg
20727,zuverge01,8,33.25,17.75,0.62,2.62,0.25,0.12,0.00,0.88,...,0.38,30.0,20.88,2.25,3.12,0.12,0.38,3.75,1.50,seasAvg
20728,zwilldu01,4,91.50,320.00,41.75,91.00,19.00,3.75,7.50,50.50,...,0.00,25.0,360.75,60.75,140.00,11.50,30.25,124.00,84.75,seasAvg


In [ ]:
   
#ratios and percentages
player_batting['BA'] = BasicHitting.calc_ba(player_batting)
player_batting['HRAB'] = BasicHitting.calc_hrab(player_batting)
player_batting['HRH'] = BasicHitting.calc_hrh(player_batting)
player_batting['OBP'] = BasicHitting.calc_obp(player_batting)
player_batting['SLGp'] = BasicHitting.calc_slg(player_batting)
player_batting['OPS'] = BasicHitting.calc_ops(player_batting)
player_batting['XBHp'] = BasicHitting.calc_xbh_pct(player_batting)
player_batting['EqA'] = BasicHitting.calc_eqa(player_batting)
player_batting['ISO'] = BasicHitting.calc_iso(player_batting)
player_batting['GPA'] = BasicHitting.calc_gpa(player_batting)
player_batting['PASO'] = BasicHitting.calc_paso(player_batting)
player_batting['BBK'] = BasicHitting.calc_bbk(player_batting)
player_batting['RC'] = BasicHitting.calc_rc(player_batting)
player_batting['BR'] = BasicHitting.calc_br(player_batting)
player_batting['SBp'] = BasicHitting.calc_sbp(player_batting)
player_batting['TA'] = BasicHitting.calc_ta(player_batting)
player_batting['BABIP'] = BasicHitting.calc_babip(player_batting)

player_batting

In [ ]:
#TODO: combine and export to csv